In [2]:
import pandas as pd
import csv

df = pd.read_csv(
    "winequality-red.csv",
    sep=";",
    engine="python",
    quoting=csv.QUOTE_NONE
)

df.columns = df.columns.str.replace('"', '')
df.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=["quality"])
y = df["quality"].astype(int)

# preprocessing (scaling)
scaler = StandardScaler()
X = scaler.fit_transform(X)   # X is now a numpy array

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# tune only the big ones: n_estimators, max_depth, max_features
n_estimators_list = [200, 400, 800]
max_depth_list = [None, 10, 20]
max_features_list = ["sqrt", "log2"]

best_mean_acc = -1
best_params = None

for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:
        for max_features in max_features_list:
            fold_acc = []

            for train_idx, val_idx in kf.split(X, y):
                X_train, X_val = X[train_idx], X[val_idx]
                y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

                model = RandomForestClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    max_features=max_features,
                    random_state=42,
                    n_jobs=-1
                )
                model.fit(X_train, y_train)

                preds = model.predict(X_val)
                fold_acc.append(accuracy_score(y_val, preds))

            mean_acc = float(np.mean(fold_acc))

            if mean_acc > best_mean_acc:
                best_mean_acc = mean_acc
                best_params = (n_estimators, max_depth, max_features)

print("Best mean 5-fold accuracy:", best_mean_acc)
print("Best params (n_estimators, max_depth, max_features):", best_params)


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# best params from your CV
best_n_estimators = 200
best_max_depth = None
best_max_features = "sqrt"

X = df.drop(columns=["quality"])
y = df["quality"].astype(int)

# final hold-out test evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

best_rf = RandomForestClassifier(
    n_estimators=best_n_estimators,
    max_depth=best_max_depth,
    max_features=best_max_features,
    random_state=42,
    n_jobs=-1
)
best_rf.fit(X_train, y_train)

y_pred = best_rf.predict(X_test)

print("Test accuracy:", accuracy_score(y_test, y_pred))



Test accuracy: 0.675
